In [3]:
import csv
import pandas as pd
import numpy as np

In [4]:
# conda create -n pd25
# conda install --force-reinstall  -n pd25 -c conda-forge requirements.txt # pandas=0.25.3
# conda activate pd25
# conda install --force-reinstall  -c conda-forge pandas=0.25.3
# conda install -c conda-forge pandas-profiling

In [5]:
print(f'{pd.__version__}') # pandas=0.25.3

1.1.4


In [6]:
# df = pd.read_csv('./data/NHIS_total_model.csv')

In [7]:
# bike_df = pd.read_csv('./bike_train.csv')
# bike_df.info()

In [8]:
df = pd.read_csv('./house_price.csv')
# df.info()
df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
# bike_df = pd.read_csv('./bike_train.csv')
# # 문자열을 datetime 타입으로 변경. 
# bike_df['datetime'] = bike_df.datetime.apply(pd.to_datetime)

# # datetime 타입에서 년, 월, 일, 시간 추출
# bike_df['year'] = bike_df.datetime.apply(lambda x : x.year)
# bike_df['month'] = bike_df.datetime.apply(lambda x : x.month)
# bike_df['day'] = bike_df.datetime.apply(lambda x : x.day)
# bike_df['hour'] = bike_df.datetime.apply(lambda x: x.hour)
# drop_columns = ['datetime','casual','registered']
# bike_df.drop(drop_columns, axis=1,inplace=True)
# # print(bike_df.info()) # 10886
# bike_df.info()

In [6]:
# df = pd.read_csv('./data/NHIS_2018_65500.csv')

In [12]:
# import pandas_profiling
# profile = bike_df.profile_report()
# profile.to_file("./report/bike_profile.html")

In [9]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df)
profile.to_file("./report/Ameshouse_profile.html")

C:\Users\PC\anaconda3\envs\pd25\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [10]:
# AmesHouse
house_df = pd.read_csv('./house_price.csv')
# Null 이 너무 많은 컬럼들과 불필요한 컬럼 삭제
house_df.drop(['Id','PoolQC' , 'MiscFeature', 'Alley', 'Fence','FireplaceQu'], axis=1 , inplace=True)
# Drop 하지 않는 숫자형 Null컬럼들은 평균값으로 대체
house_df.fillna(house_df.mean(),inplace=True)
null_column_count = house_df.isnull().sum()[house_df.isnull().sum() > 0]
# print('## Null 피처의 Type :\n', house_df.dtypes[null_column_count.index])
# 문자형(범주형) feature one-hot 인코딩으로 shape 변환됨 (1460, 75) ==> (1460, 271) 
house_df_ohe = pd.get_dummies(house_df)
null_column_count = house_df_ohe.isnull().sum()[house_df_ohe.isnull().sum() > 0]
# print('## Null 피처의 Type :\n', house_df_ohe.dtypes[null_column_count.index])
## scaler2 ==> X_features skew큰것 log1p변환
from scipy.stats import skew
# object가 아닌 숫자형 피쳐의 컬럼 index 객체 추출.
features_index = house_df.dtypes[house_df.dtypes != 'object'].index
# house_df에 컬럼 index를 [ ]로 입력하면 해당하는 컬럼 데이터 셋 반환. apply lambda로 skew( )호출 
skew_features = house_df[features_index].apply(lambda x : skew(x))
# skew 정도가 1 이상인 컬럼들만 추출. 
skew_features_top = skew_features[skew_features > 1]
print(skew_features_top.sort_values(ascending=False))
house_df[skew_features_top.index] = np.log1p(house_df[skew_features_top.index])
house_df_ohe = pd.get_dummies(house_df)

MiscVal          24.451640
PoolArea         14.813135
LotArea          12.195142
3SsnPorch        10.293752
LowQualFinSF      9.002080
KitchenAbvGr      4.483784
BsmtFinSF2        4.250888
ScreenPorch       4.117977
BsmtHalfBath      4.099186
EnclosedPorch     3.086696
MasVnrArea        2.673661
LotFrontage       2.382499
OpenPorchSF       2.361912
SalePrice         1.880941
BsmtFinSF1        1.683771
WoodDeckSF        1.539792
TotalBsmtSF       1.522688
MSSubClass        1.406210
1stFlrSF          1.375342
GrLivArea         1.365156
dtype: float64


In [12]:
from pandas_profiling import ProfileReport
profile = ProfileReport(house_df_ohe)
profile.to_file("./report/Ameshouse_scaled_profile.html")

C:\Users\PC\anaconda3\envs\pd25\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)
